# How to download feedback and examples from a test project
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langsmith-cookbook/blob/master/./testing-examples/download-feedback-and-examples/download_example.ipynb)

When testing with Langsmith, all the traces, examples, and evaluation feedback are saved so you have a full audit of what happened.
This way you can see the aggregate metrics of the test run and compare on an example by example basis. You can also download the run and evaluation result information
to use in external reporting software.

In this walkthrough, we will show how to export the feedback and examples from a Langsmith test project. The main steps are:

1. Create a dataset
2. Run testing
3. Export feedback and examples

## Setup

Install langchain and any other dependencies for your chain. We will install pandas as well for this walkthrough to put the retrieved data in a dataframe.

In [ ]:
# %pip install -U langsmith langchain anthropic pandas --quiet

In [1]:
import uuid
import os

unique_id = uuid.uuid4().hex[0:8]
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_ENDPOINT"] = ""
os.environ["LANGCHAIN_PROJECT"] = f"Retrieve feedback and examples {unique_id}"

## 1. Create a dataset

We will create a simple KV dataset with a poem topic and a constraint letter (which the model should not use).

In [2]:
from langsmith import Client

client = Client()

examples= [
    ("roses", "o"),
    ("vikings", "v"),
    ("planet earth", "e"),
    ("Sirens of Titan", "t"),
]

dataset_name = f"Download Feedback and Examples {unique_id}"
dataset = client.create_dataset(dataset_name)

for prompt, constraint in examples:
    client.create_example({"input": prompt, "constraint": constraint}, dataset_id=dataset.id, outputs={"constraint": constraint})

## 2. Run testing

We will use a simple custom evaluator that checks whether the prediction contains the constraint letter.

In [3]:
from typing import Any
from langchain.evaluation import StringEvaluator

class ConstraintEvaluator(StringEvaluator):
    
    @property
    def requires_reference(self):
        return True
    
    def _evaluate_strings(self, prediction: str, reference: str, **kwargs: Any) -> dict:
        # Reference in this case is the letter that should not be present
        return {
            "score": bool(reference not in prediction),
            "reasoning": f"prediction contains the letter {reference}",
        }

In [4]:
from langchain import chat_models, prompts, schema
from langchain.smith import RunEvalConfig

chain = (
    prompts.PromptTemplate.from_template("Write a poem about {input} without using the letter {constraint}. Respond directly with the poem with no explanation.")
    | chat_models.ChatAnthropic()
    | schema.output_parser.StrOutputParser()
)

eval_config = RunEvalConfig(
    custom_evaluators=[ConstraintEvaluator()],
    input_key="input",
)

test_results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=eval_config,
)

View the evaluation results for project 'test-kind-prose-61' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/9890dcfa-92af-42d0-9df2-aa3dccd13788
[------------------------------------------------->] 4/4

## 3. Review the feedback and examples

If you want to directly use the results, you can easily access them in tabular format by calling `to_dataframe()` on the test_results. 

In [5]:
test_results.to_dataframe()

,ConstraintEvaluator,input,output,reference
3e9a2c05-f7f5-4309-b755-abeb76719f26,False,"{'input': 'Sirens of Titan', 'constraint': 't'}",Here is a poem about Sirens of Titan without ...,{'constraint': 't'}
9fddeee7-9080-46c8-b280-405b4f3d18cb,False,"{'input': 'planet earth', 'constraint': 'e'}","Our orb of life, a vision grand,\nWith oceans...",{'constraint': 'e'}
d74ea995-6ae6-488a-8da8-21695f8949fe,False,"{'input': 'vikings', 'constraint': 'v'}",Here is a poem about Vikings without using th...,{'constraint': 'v'}
e059d4b8-3079-40c7-a6b7-16347d4fc568,False,"{'input': 'roses', 'constraint': 'o'}",Here is a poem about roses without using the ...,{'constraint': 'o'}


If you want to fetch the feedback and examples for a historic test project, you can use the SDK:

In [ ]:
# Can be any previous test projects
test_project = test_results['project_name']

In [ ]:
import pandas as pd

runs = client.list_runs(project_name=test_project, execution_order=1)

df = pd.DataFrame(
    [
        {
            "example_id": r.reference_example_id,
            **r.inputs,
            **(r.outputs or {}),
            **{k: v for f in client.list_feedback(run_ids=[r.id]) for k, v in [(f"{f.key}.score", f.score), (f"{f.key}.comment", f.comment)]},
            "reference": client.read_example(r.reference_example_id).outputs
        }
        for r in runs
    ]
)
df

## Conclusion

In this example we showed how to download feedback and examples from a test project. You can directly use the result object from the run or use the SDK to fetch the results and feedback.
Use this to analyze further or to programmatically add result information to your existing reports.